### 1. 文件读取

In [62]:
import os
import numpy as np
import scipy.stats as st
filepath = '../../Data/'

pathDir =  os.listdir(filepath)
data_index = [] #所有数据集名字列表，不带路径，不带扩展名
for i in pathDir:
    if (i[i.rfind('.'):] == '.npz'):
        data_name = i[:-4]
        data_index.append(data_name)
print("There are {0} \'.npz\' file in the path!\nThey are {1}".format(len(data_index), data_index))



There are 26 '.npz' file in the path!
They are ['Aiden_GM12878_Rep2_500000', 'Aiden_IMR90_Rep2_500000', 'Aiden_K562_500000', 'BIRM_BEP2D_500000', 'BIRM_BERP35T1_500000', 'BIRM_BERP35T4_500000', 'Dekker_A549_500000', 'Dekker_CAKI2_500000', 'Dekker_G401_500000', 'Dekker_LNCaP_500000', 'Dekker_NCIH460_500000', 'Dekker_PANC1_500000', 'Dekker_RPMI7951_500000', 'Dekker_SJCRH30_500000', 'Dekker_SKMEL5_500000', 'Dekker_SKNDZ_500000', 'Dekker_SK-N-MC_500000', 'Dekker_T47D_500000', 'Farnham_22RV1_500000', 'Farnham_C42B_500000', 'Farnham_RWPE1_500000', 'Lee_RPMI8226_H_500000', 'Lee_RPMI8226_M_500000', 'Lee_U266_H_500000', 'Lee_U266_M_500000', 'RenLab_IMR90_500000']


### 2. 子函数定义

In [63]:
def in_put_npz(filepath,filename):
    file = filepath + filename + '.npz'
    data = np.load(file)
    return data

def pre_deal_dataset(data):#data是load的npz数据,获得数据集中不同的染色体位置
    pos = np.zeros(25,dtype = np.int64) #对应1-22号染色体和X，Y染色体位置，最后一位是Y染色体终止位
    for i in range(1,25): #找出不同染色体区域
        if i<23:        
            pos[i-1] = np.where(data['col'] == 'chr'+str(i)+'-0')[0][0]
        else:
            if i == 23:
                pos[i-1] = np.where(data['col'] == 'chrX-0')[0][0]
            else:
                pos[i-1] = np.where(data['col'] == 'chrY-0')[0][0]
    pos[-1] = 6206 #Y终止位
    diff = np.diff(pos) #每条染色体的矩阵规模
    return pos,diff

def chr_interaction_pick(data,i,j):#输入总的数据集，输出想要比对的chr_i,chr_j构成的染色体间互作矩阵
    pos,size = pre_deal_dataset(data)
    return data['hic'][pos[i]:pos[i]+size[i],pos[j]:pos[j]+size[j]]

def calcu_chr_interaction(data,i,j):# 输入总的数据集，想要对比的chr_i,chr_j，输出能量值分布 分布峰度
    M = chr_interaction_pick(data,i,j)
    cnt = 3
    while cnt>0:
        M[np.where(M==np.amax(M))] = 0
        cnt = cnt - 1
        
    X1 = np.dot(M,M.T)
    X2 = np.dot(M.T,M)
    H1,bin_edges1 = np.histogram(np.amax(X1,1),bins='auto',density = True)
    H2,bin_edges2 = np.histogram(np.amax(X2,1),bins='auto',density = True)
    energy = np.dot(H1,bin_edges1[0:-1])+np.dot(H2,bin_edges2[0:-1])
    kurt = st.kurtosis(np.amax(X1,1))+st.kurtosis(np.amax(X2,1))
    return energy,kurt
    
    

def spectral(S):#输入对称方阵
    eigenvalue,eigenvector = np.linalg.eigh(S,UPLO='L')
    x= eigenvalue.real
    y= eigenvalue.imag
    z = abs(eigenvalue)
    
    


### 3. 主运行程序 

In [65]:
for filename in data_index:
#     filename = data_index[2]
    data = in_put_npz(filepath,filename)
    print(filename)
    
    value_struct = np.dtype([('pos_i','i'),('pos_j','i'),('energy','f'),('kurt','f')])
    len_value_L = 23*24/2
    value_L = np.zeros((np.int(len_value_L),),dtype = value_struct)
    k = 0
    for i in range(0,24):
        for j in range(i+1,24):
            tmp_energy,tmp_kurt = calcu_chr_interaction(data,i,j)
#             print(i,j,tmp_ans)
            value_L[k]=(i,j,-tmp_energy,-tmp_kurt)
            k = k + 1
    print(filename,':',np.sort(value_L,order='kurt'))

    

Aiden_GM12878_Rep2_500000
Aiden_GM12878_Rep2_500000 : [( 0, 23,  -52.02075  , -3.67349365e+02)
 ( 0,  3,  -17.15783  , -3.53138947e+02)
 ( 5, 15,   -8.888488 , -2.52133163e+02)
 ( 0, 20,   -9.766791 , -1.73153427e+02)
 ( 0, 15,   -9.80746  , -1.49838638e+02)
 ( 3, 23,  -22.668364 , -1.40536423e+02)
 (13, 21,   -5.347302 , -1.14832779e+02)
 ( 0,  1,  -11.743742 , -1.05402718e+02)
 ( 3, 18,  -12.68822  , -9.42838516e+01)
 ( 3, 20,   -7.5413585, -8.99664612e+01)
 (20, 23,   -1.943169 , -8.41302490e+01)
 (22, 23, -395.55255  , -7.04066696e+01)
 (15, 23,   -2.0873764, -6.87817383e+01)
 (13, 23,   -1.9316006, -6.24689751e+01)
 (14, 23,   -2.3794158, -5.63319321e+01)
 (21, 23,   -1.4962878, -5.45137329e+01)
 (12, 23,  -20.625158 , -5.38706856e+01)
 (16, 23,   -1.4780121, -5.32374840e+01)
 (17, 23,   -2.1364877, -4.99261398e+01)
 ( 8, 23,   -5.9777594, -4.90558739e+01)
 (18, 23,   -1.7142857, -4.65714607e+01)
 ( 2, 23,   -9.171581 , -4.57551880e+01)
 (19, 23,   -2.7654338, -4.56250725e+01)
 ( 

In [ ]:
filename = data_index[2]
data = in_put_npz(filepath,filename)
print(filename)
    
energy_struct = np.dtype([('pos_i','i'),('pos_j','i'),('energy','f')])
len_energy_L = 23*24/2
energy_L = np.zeros((np.int(len_energy_L),),dtype = energy_struct)
k = 0
for i in range(0,24):
    for j in range(i+1,24):
        tmp_ans = calcu_chr_interaction(data,i,j)
        print(i,j,tmp_ans)
        energy_L[k]=(i,j,tmp_ans)
        k = k + 1
    
print(filename,':',energy_L)
    

In [57]:
energy_struct = np.dtype([('pos_i','i'),('pos_j','i'),('energy','f')])
len_energy_L = 23*24/2
energy_L = np.zeros((np.int(len_energy_L),),dtype = energy_struct)
# for p in range(0,2):
k = 0
for i in range(0,24):
    for j in range(i+1,24):
#         tmp_ans = calcu_chr_interaction(data,i,j)
#         print(i,j,0.1)
        energy_L[k]=(i,j,0.1*k*(-1))
        k = k + 1
    #         print(k)
    out = np.sort(energy_L,order='energy')
print(energy_L,out)

[( 0,  1,  -0. ) ( 0,  2,  -0.1) ( 0,  3,  -0.2) ( 0,  4,  -0.3)
 ( 0,  5,  -0.4) ( 0,  6,  -0.5) ( 0,  7,  -0.6) ( 0,  8,  -0.7)
 ( 0,  9,  -0.8) ( 0, 10,  -0.9) ( 0, 11,  -1. ) ( 0, 12,  -1.1)
 ( 0, 13,  -1.2) ( 0, 14,  -1.3) ( 0, 15,  -1.4) ( 0, 16,  -1.5)
 ( 0, 17,  -1.6) ( 0, 18,  -1.7) ( 0, 19,  -1.8) ( 0, 20,  -1.9)
 ( 0, 21,  -2. ) ( 0, 22,  -2.1) ( 0, 23,  -2.2) ( 1,  2,  -2.3)
 ( 1,  3,  -2.4) ( 1,  4,  -2.5) ( 1,  5,  -2.6) ( 1,  6,  -2.7)
 ( 1,  7,  -2.8) ( 1,  8,  -2.9) ( 1,  9,  -3. ) ( 1, 10,  -3.1)
 ( 1, 11,  -3.2) ( 1, 12,  -3.3) ( 1, 13,  -3.4) ( 1, 14,  -3.5)
 ( 1, 15,  -3.6) ( 1, 16,  -3.7) ( 1, 17,  -3.8) ( 1, 18,  -3.9)
 ( 1, 19,  -4. ) ( 1, 20,  -4.1) ( 1, 21,  -4.2) ( 1, 22,  -4.3)
 ( 1, 23,  -4.4) ( 2,  3,  -4.5) ( 2,  4,  -4.6) ( 2,  5,  -4.7)
 ( 2,  6,  -4.8) ( 2,  7,  -4.9) ( 2,  8,  -5. ) ( 2,  9,  -5.1)
 ( 2, 10,  -5.2) ( 2, 11,  -5.3) ( 2, 12,  -5.4) ( 2, 13,  -5.5)
 ( 2, 14,  -5.6) ( 2, 15,  -5.7) ( 2, 16,  -5.8) ( 2, 17,  -5.9)
 ( 2, 18,  -6. ) ( 2, 19,